Companies often receive thousands of resumes for each job posting and employ dedicated screening officers to screen qualified candidates.Hiring the right talent is a challenge for all businesses.

 This challenge is magnified by the high volume of applicants if the business is labour-intensive, growing, and facing high attrition rates.IT departments are short of growing markets.

  In a typical service organization, professionals with a variety of technical skills and business domain expertise are hired and assigned to projects to resolve customer issues.

  This task of selecting the best talent among many others is known as Resume Screening.Typically, large companies do not have enough time to open each CV, so they use machine learning algorithms for the Resume Screening task.

#**IMPORTING THE NECESSARY LIBRARIES**

In [27]:
import pandas as pd
import re
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords as stp
from sklearn.feature_extraction.text import CountVectorizer
import plotly.express as px
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
import string
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [28]:
df = pd.read_csv('/content/UpdatedResumeDataSet.csv')
df_cp = df.copy()

In [29]:
df.head()

,Category,Resume
0,Data Science,Skills * Programming Languages: Python (pandas...
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...
2,Data Science,"Areas of Interest Deep Learning, Control Syste..."
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab..."


In [30]:
df.isnull().sum()

Category    0
Resume      0
dtype: int64

In [31]:
df.shape

(962, 2)

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 962 entries, 0 to 961
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  962 non-null    object
 1   Resume    962 non-null    object
dtypes: object(2)
memory usage: 15.2+ KB


In [33]:
fig = px.histogram(
    df, x="Category", title="Distribution of Jobs Categories"
).update_xaxes(categoryorder="total descending")
fig.show()

In [34]:
series = df['Category'].value_counts()

df_result = pd.DataFrame(series)

df_result = df_result.reset_index()

df_result.columns = ['Category', 'Total']

print(df_result)

                     Category  Total
0              Java Developer     84
1                     Testing     70
2             DevOps Engineer     55
3            Python Developer     48
4               Web Designing     45
5                          HR     44
6                      Hadoop     42
7                  Blockchain     40
8               ETL Developer     40
9          Operations Manager     40
10               Data Science     40
11                      Sales     40
12        Mechanical Engineer     40
13                       Arts     36
14                   Database     33
15     Electrical Engineering     30
16         Health and fitness     30
17                        PMO     30
18           Business Analyst     28
19           DotNet Developer     28
20         Automation Testing     26
21  Network Security Engineer     25
22              SAP Developer     24
23             Civil Engineer     24
24                   Advocate     20


In [35]:
fig = px.pie(df_result,
                 values='Total',
                 names='Category')

fig.show()

#**WE ARE GOING TO ONLY USE THE RESUME THAT RELATED TO JAVA DEVELOPER ROLE.**

In [36]:


# Filter resumes for the 'Java Developer' role
java_developer_resumes = df_cp[df_cp['Category'] == 'Java Developer']

# Display the extracted resumes
print(f"Total Java Developer Resumes: {len(java_developer_resumes)}")
print(java_developer_resumes[['Category', 'Resume']])  # Displaying only 'Category' and 'Resume' columns

# Optional: Save the filtered resumes to a new CSV file
java_developer_resumes.to_csv('/content/JavaDeveloperResumes.csv', index=False)


Total Java Developer Resumes: 84
           Category                                             Resume
319  Java Developer  Education Details \r\nAugust 2010 to May 2017 ...
320  Java Developer  Technical Skills (Trained / Project Acquired S...
321  Java Developer  TECHNICAL SKILLS Skills: Java, SQL, PL/SQL, C,...
322  Java Developer  TECHNICAL STRENGTHS Computer Language Java/J2E...
323  Java Developer  Education Details \r\nJanuary 2013 Master of E...
..              ...                                                ...
398  Java Developer  Computer Skills: Languages And Script: JSP, Se...
399  Java Developer  Education Details \r\nJanuary 2016 B.E Informa...
400  Java Developer  TECHNICAL SKILLS Programming Languages: Java (...
401  Java Developer  SKILLS: - 1) Team leading 2) Self-motivated 3)...
402  Java Developer  Skills â¢ Language: Java â¢ Operating System...

[84 rows x 2 columns]


In [37]:
df_java = pd.read_csv('/content/JavaDeveloperResumes.csv')
df_java.head(10)


,Category,Resume
0,Java Developer,Education Details \r\nAugust 2010 to May 2017 ...
1,Java Developer,Technical Skills (Trained / Project Acquired S...
2,Java Developer,"TECHNICAL SKILLS Skills: Java, SQL, PL/SQL, C,..."
3,Java Developer,TECHNICAL STRENGTHS Computer Language Java/J2E...
4,Java Developer,Education Details \r\nJanuary 2013 Master of E...
5,Java Developer,Education Details \r\n BE IT pjlce\r\nJava D...
6,Java Developer,"TECHNICALSKILLS SpringMVC, Hibernate, JDBC, Ja..."
7,Java Developer,"TECHNICALSKILLS SpringMVC, Hibernate, JDBC, Ja..."
8,Java Developer,"Operating Systems Windows XP, 7, 10. Tools/Pac..."
9,Java Developer,"Computer Skills: Languages And Script: JSP, Se..."


In [38]:
def cleanResume(resumeText):
    resumeText = re.sub('http\S+\s*', ' ', resumeText)  # remove URLs
    resumeText = re.sub('RT|cc', ' ', resumeText)  # remove RT and cc
    resumeText = re.sub('#\S+', '', resumeText)  # remove hashtags
    resumeText = re.sub('@\S+', '  ', resumeText)  # remove mentions
    resumeText = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', resumeText)  # remove punctuations
    resumeText = re.sub(r'[^\x00-\x7f]',r' ', resumeText)
    resumeText = re.sub('\s+', ' ', resumeText)  # remove extra whitespace

    return resumeText

df_java['cleaned_resume'] = df_java.Resume.apply(lambda x: cleanResume(x))

In [60]:
df_java.head()

,Category,Resume,cleaned_resume,skills
0,Java Developer,Education Details \r\nAugust 2010 to May 2017 ...,Education Details August 2010 to May 2017 BE E...,[]
1,Java Developer,Technical Skills (Trained / Project Acquired S...,Technical Skills Trained Project Acquired Skil...,[]
2,Java Developer,"TECHNICAL SKILLS Skills: Java, SQL, PL/SQL, C,...",TECHNICAL SKILLS Skills Java SQL PL SQL C C Bo...,[]
3,Java Developer,TECHNICAL STRENGTHS Computer Language Java/J2E...,TECHNICAL STRENGTHS Computer Language Java J2E...,[]
4,Java Developer,Education Details \r\nJanuary 2013 Master of E...,Education Details January 2013 Master of Engin...,[]


In [40]:
import spacy
from spacy.pipeline import EntityRuler
from spacy.lang.en import English
from spacy.tokens import Doc


In [41]:
!python -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 9.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [61]:
# Function to extract key information from resumes
def extract_resume_info(resume_text):
    doc = nlp(resume_text)
    experience = []
    skills = []
    education = []
    certifications = []
    for entity in doc.ents:
        if entity.label_ == "DATE" or entity.label_ == "TIME":
            experience.append(entity.text)
        elif entity.label_ == "SKILL":
            skills.append(entity.text)
        elif entity.label_ == "EDUCATION":
            education.append(entity.text)
        elif entity.label_ == "CERTIFICATION":
            certifications.append(entity.text)

    return experience, skills, education,certifications

In [75]:

def rank_candidates(job_description, candidates):
    vectorizer = TfidfVectorizer(stop_words='english')
    job_description_vector = vectorizer.fit_transform([job_description])

    candidate_vectors = []
    for candidate in candidates:
        candidate_vector = vectorizer.transform([candidate])
        candidate_vectors.append(candidate_vector)

    similarities = [cosine_similarity(job_description_vector, vec)[0][0] for vec in candidate_vectors]
    ranked_candidates = sorted(zip(candidates, similarities), key=lambda x: x[1], reverse=True)

    return ranked_candidates

job_description = """
Skills :
** Proficiency in programming languages – Java
** Strong in Algorithms and Data Structures
** Strong software design, profiling, and verification skills
** Strong in fundamentals - High-performance Computing, Event-based programming, Multi-threading, Memory optimization, Fault Tolerance, etc.
** Sound knowledge of FreeBSD or any Unix systems internals
** Good understanding of distributed systems
** Very good debugging and troubleshooting skills
** Proficient in design/code reviews and preferably in writing automatable unit and integration tests
** Excellent verbal, written and interpersonal skills
** Self-starter takes initiative & ownership, identifies and completes tasks; goal-driven
** Ability to work with remote teams, handle multiple tasks, prioritize and meet deadlines

"""



# Extract information from resumes in the DataFrame
candidates_info = [extract_resume_info(resume) for resume in df_java['cleaned_resume']]

# Rank candidates based on job description
ranked_candidates = rank_candidates(job_description, df_java['cleaned_resume'].tolist())

# Display ranked candidates
for candidate, similarity in ranked_candidates:
    print(f"Similarity: {similarity:.2%}")
    print(candidate)
    print("*"*50)





Similarity: 50.17%
TECHNICAL SKILLS Skills Java SQL PL SQL C C BootStrap JSP Ext JS Operating Systems Windows Tools Toad Eclipse SoapBox Postman Databases Oracle MS SQL MS A ess MS ExcelEducation Details January 2017 B E Computer Technology Gondia MAHARASHTRA IN Manoharbhai Patel Institute of Engineering and Technology HSC Gondia MAHARASHTRA IN S M P Science College Java Developer Java Developer Xoriant Solutions Pvt Ltd Skill Details Company Details company Xoriant Solutions Pvt Ltd description Pune Period Sept 2017 to till date Role Java Developer Description ResRent is a team responsible for the development of various modules for Hertz They are responsible for the development of screens in the portfolios they look after and act as a primary control function developing webservices to ensure it can be hit by XML and JSON We build various modules which are inbuilt in Hertz to provide interfaces for end users in their tasks Tools Eclipse TOAD SoapBox Postman Responsibilities Develop mod

In [76]:

def rank_candidates(job_description, candidates):
    vectorizer = TfidfVectorizer(stop_words='english')
    job_description_vector = vectorizer.fit_transform([job_description])

    candidate_vectors = []
    for candidate in candidates:
        candidate_vector = vectorizer.transform([candidate])
        candidate_vectors.append(candidate_vector)

    similarities = [cosine_similarity(job_description_vector, vec)[0][0] for vec in candidate_vectors]
    ranked_candidates = sorted(zip(candidates, similarities), key=lambda x: x[1], reverse=True)

    return ranked_candidates

job_description = """
Skills :
** Excellent communication skills (verbal and written)
** Quick learner, with a positive attitude and ability to work well within a team.
** Understanding of database concepts such as tables, queries, and joins.
** Awareness of Extract, Transform, Load processes and their importance in data engineering.
** Basic proficiency in a programming language like Python, Java, or Scala.
** Basic understanding of data modeling techniques and database design principles.
** Familiarity with cloud platforms like AWS, Azure, or Google Cloud Platform.
** Basic knowledge of version control systems like Git for managing codebase changes
"""



# Extract information from resumes in the DataFrame
candidates_info = [extract_resume_info(resume) for resume in df_java['cleaned_resume']]

# Rank candidates based on job description
ranked_candidates = rank_candidates(job_description, df_java['cleaned_resume'].tolist())

# Display ranked candidates
for candidate, similarity in ranked_candidates:
    print(f"Similarity: {similarity:.2%}")
    print(candidate)
    print("*"*50)





Similarity: 36.13%
TECHNICAL SKILLS Skills Java SQL PL SQL C C BootStrap JSP Ext JS Operating Systems Windows Tools Toad Eclipse SoapBox Postman Databases Oracle MS SQL MS A ess MS ExcelEducation Details January 2017 B E Computer Technology Gondia MAHARASHTRA IN Manoharbhai Patel Institute of Engineering and Technology HSC Gondia MAHARASHTRA IN S M P Science College Java Developer Java Developer Xoriant Solutions Pvt Ltd Skill Details Company Details company Xoriant Solutions Pvt Ltd description Pune Period Sept 2017 to till date Role Java Developer Description ResRent is a team responsible for the development of various modules for Hertz They are responsible for the development of screens in the portfolios they look after and act as a primary control function developing webservices to ensure it can be hit by XML and JSON We build various modules which are inbuilt in Hertz to provide interfaces for end users in their tasks Tools Eclipse TOAD SoapBox Postman Responsibilities Develop mod